In [1]:
import numpy as np 
import time 
import numpy as np
import torch
import torch.nn as nn
import time
import gym 
import pickle 
from replay_memory import Memory
import argparse
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.distributions.transforms import TanhTransform
import tqdm

from nets import Actor_D

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
EPS = 1e-7

In [4]:
env_name='Ant-v3'
data_path = "/home/ns/bc_tutorial/mujoco/expert_data/Ant-v3_10_3765.pkl"

In [5]:
with open(data_path, "rb") as f:
    data_good = pickle.load(f)
print('expert data loaded')

data_good=data_good[:20]

good_obs=[]
good_acts=[]
good_rews=[]

for traj in data_good: 
    s,a,r=traj  

    good_obs.append(s)
    good_acts.append(a)
    good_rews.append(r)

good_obs=np.vstack(good_obs)
good_acts=np.vstack(good_acts)
good_rews=np.vstack(good_rews) 

good_obs.shape, good_acts.shape, good_rews.shape

expert data loaded


((10000, 111), (10000, 8), (10000, 1))

In [6]:
memory=Memory(maxlen=1000000)
memory.add_data(good_obs, good_acts, good_rews)

In [7]:
seed=0
torch.manual_seed(seed)
np.random.seed(seed) 
env = gym.make(env_name) 

ob_dim = env.observation_space.shape[0]
ac_dim = env.action_space.shape[0]

actor = Actor_D(ac_dim, ob_dim).to(device)
 
optimizer = torch.optim.Adam(actor.parameters(), 0.0001)

/home/ns/anaconda3/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Ant-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/home/ns/anaconda3/lib/python3.9/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


In [8]:
verbose=True
batch_size=100
training_timestep=10000
actor.train()
for train_step in range(1, 50000+1):  
    batch_states, batch_actions, batch_rewards=memory.sample_batch(batch_size)  
    batch_states = torch.Tensor(batch_states).to(device)
    batch_actions=torch.Tensor(batch_actions).to(device)
     
    log_pi_e = actor.get_log_density(batch_states, batch_actions) 
    bc_loss = -torch.sum(log_pi_e, 1)
    loss = torch.mean(bc_loss)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if verbose and (train_step % (training_timestep/10)==0):
        print('step:', train_step, ' loss', loss.item()) 

print('bc training completed')

step: 1000  loss -4.902913570404053
step: 2000  loss -6.244664669036865
step: 3000  loss -7.190088748931885
step: 4000  loss -7.46046257019043
step: 5000  loss -7.884944915771484
step: 6000  loss -7.924429893493652
step: 7000  loss -8.098227500915527
step: 8000  loss -8.21967887878418
step: 9000  loss -8.855413436889648
step: 10000  loss -8.62328815460205
step: 11000  loss -9.741211891174316
step: 12000  loss -8.712021827697754
step: 13000  loss -8.523443222045898
step: 14000  loss -9.51380443572998
step: 15000  loss -9.348519325256348
step: 16000  loss -9.608366012573242
step: 17000  loss -9.730209350585938
step: 18000  loss -9.397101402282715
step: 19000  loss -10.095282554626465
step: 20000  loss -9.801156044006348
step: 21000  loss -10.115110397338867
step: 22000  loss -10.215883255004883
step: 23000  loss -10.161785125732422
step: 24000  loss -9.732927322387695
step: 25000  loss -9.744319915771484
step: 26000  loss -10.043241500854492
step: 27000  loss -10.586008071899414
step: 28

In [14]:
def play(env, policy, is_close=True, is_render=True, max_step=1000): 
    obs,info = env.reset()
    dones=False
    total_r=0
    step=0
    while not dones: 
        step+=1
        obs=torch.Tensor(obs[None]).to(device)
        # action, logp_pi, a_tanh_mode = actor(obs)
        action=actor.get_mean(obs)
        action=action.cpu().detach().numpy() 

        obs, rewards, done, s, info = env.step(action.ravel())
        total_r +=rewards  
        if done:
            break
        if step>max_step:
            # print('max step reached')
            break
        # elif s:
        #     print('solved!')
        #     break
    if is_close:
        env.close()
    return {'reward':total_r, 'step':step-1}

In [10]:
env_name='Ant-v3' 
env = gym.make(env_name)

/home/ns/anaconda3/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Ant-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/home/ns/anaconda3/lib/python3.9/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


In [11]:
env = gym.make(env_name)
play(env, actor, is_close=True, is_render=False)

{'reward': 4121.5574559161805, 'step': 1000}

In [12]:
rewards=[]
for i in tqdm.tqdm( range(50) ):
    stats=play(env, actor, is_close=True, is_render=False)
    rewards.append(stats['reward'])
    # print(f'i={i} reward={stats["reward"]} step={stats["step"]}')
print(f'mean reward={np.mean(rewards) :.2f} std={np.std(rewards):.2f}')

100%|██████████| 50/50 [00:38<00:00,  1.31it/s]

mean reward=3967.68 std=418.52


In [13]:
print(f'mean reward={np.mean(rewards) :.2f} std={np.std(rewards):.2f}')

mean reward=3967.68 std=418.52
